In [ ]:
# %pip install -q wandb
# 깔끔하게 설치

In [8]:
import wandb
wandb.login()

wandb: Currently logged in as: goeun. Use `wandb login --relogin` to force relogin


True

In [13]:
# %load train_lib.py

def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = {
        'layers': 128,
        'activation': 'relu'
    }
    wandb.init(project="sweep-intro", config=config_defaults, magic=True)

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images.shape
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(wandb.config.layers, activation=wandb.config.activation),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5,
                  validation_data=(test_images, test_labels))


In [14]:
sweep_config = {
    'project': 'sweep-intro',
    'method': 'grid',
    'parameters': {
        'layers': {
            'values': [32, 64, 96, 128, 256]
        },
        'activation': {
            'values': ['relu', 'sigmoid', 'tanh']
        }

    }
}

In [15]:
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 1k5gf5h8
Sweep URL: https://wandb.ai/goeun/sweep-intro/sweeps/1k5gf5h8


In [17]:
wandb.agent(sweep_id, function=train)

GFLOPs,5e-05


In [18]:
wandb.finish()